# **SVM**
- Using data that was annotated
- Training it on the unbalanced data
- Using avgpool
- 1 second window
- Entropy is the measure of information contained in a state


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random

In [25]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_1s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [26]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [27]:
train_data

{'melspectrogram': array([[-0.01898465, -0.2811592 , -0.25845313, ..., -0.9981291 ,
         -0.9983617 , -1.        ],
        [-0.01670736, -0.27276382, -0.2651618 , ..., -0.9903572 ,
         -0.9974626 , -1.        ],
        [-0.01776626, -0.26800779, -0.26286083, ..., -0.9897578 ,
         -0.9985065 , -0.9999572 ],
        ...,
        [-0.07289265, -0.5997221 , -0.6325296 , ..., -0.9177227 ,
         -0.93674475, -0.99452364],
        [-0.01720924, -0.9334746 , -0.9393428 , ..., -0.8331133 ,
         -0.8683263 , -0.98977023],
        [-0.06566753, -0.6877171 , -0.70655054, ..., -0.8833477 ,
         -0.89263326, -0.9825436 ]], dtype=float32),
 'mfcc': array([[-9.9767429e-01,  5.7248688e-01, -2.8398493e-01, ...,
          1.3116055e-02,  6.1448671e-02,  3.6781926e-02],
        [-9.9636453e-01,  5.7445627e-01, -2.7706087e-01, ...,
          2.2723455e-02,  6.6032313e-02,  2.8396677e-02],
        [-1.0000000e+00,  5.6324875e-01, -2.5171626e-01, ...,
          3.5349183e-02,  6.71

In [28]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [29]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [30]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [31]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [32]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [33]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [34]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 128)

array([-0.01743852, -0.6508722 , -0.6916726 , -0.63577497, -0.6325771 ,
       -0.693267  , -0.69924164, -0.7079409 , -0.73118097, -0.7504752 ,
       -0.78667325, -0.7978573 , -0.80476624, -0.8203451 , -0.7949259 ,
       -0.789861  , -0.8088392 , -0.82532144, -0.83248717, -0.80361825,
       -0.83153164, -0.85650146, -0.85409397, -0.85105276, -0.8633566 ,
       -0.8759206 , -0.8782255 , -0.8650762 , -0.8831901 , -0.88035285,
       -0.8797179 , -0.8726353 , -0.870042  , -0.86516607, -0.84496367,
       -0.82393944, -0.798188  , -0.7757094 , -0.76359135, -0.7636499 ,
       -0.7554068 , -0.7708649 , -0.7789858 , -0.7851823 , -0.8036501 ,
       -0.82371676, -0.8172532 , -0.7944738 , -0.7687108 , -0.7613045 ,
       -0.73363197, -0.7210863 , -0.73323107, -0.7296212 , -0.7292174 ,
       -0.73931384, -0.76318365, -0.7856355 , -0.7871114 , -0.7552131 ,
       -0.7135708 , -0.67738   , -0.6638213 , -0.6613151 , -0.6510076 ,
       -0.67116624, -0.7190023 , -0.7187137 , -0.65514815, -0.60

'mfcc'

(12565, 20)

...

(12565, 84)

array([-29.39905 , -28.323952, -30.232784, -29.47406 , -27.153494,
       -24.825844, -26.720036, -26.476421, -28.381443, -35.24852 ,
       -35.75719 , -36.161526, -38.84649 , -38.180325, -36.2371  ,
       -36.643337, -38.62381 , -40.752415, -39.763824, -33.694458,
       -34.624397, -36.676933, -29.563038, -31.875635, -36.143166,
       -39.61375 , -39.852917, -40.979412, -41.2093  , -41.830776,
       -41.238277, -40.43606 , -42.073097, -43.597797, -43.47115 ,
       -44.162884, -46.430286, -48.147423, -48.957855, -49.706734,
       -50.141304, -51.65557 , -51.007893, -49.478077, -49.370472,
       -50.613445, -52.21544 , -52.10996 , -50.735115, -53.456318,
       -54.24676 , -55.094383, -56.596664, -56.408787, -55.866055,
       -57.131966, -56.22503 , -54.807167, -53.670033, -52.506256,
       -51.82573 , -51.714436, -53.293312, -53.486908, -52.728363,
       -51.071804, -51.70296 , -52.312763, -51.750168, -50.660168,
       -46.681023, -46.31015 , -45.479893, -43.51856 , -41.098

'id'

(12565,)

'XC483906.mp3'

In [35]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [36]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 128)

array([-0.01707026, -0.39656842, -0.4183283 , -0.45894244, -0.49384403,
       -0.51980877, -0.52839357, -0.5510067 , -0.58286625, -0.5928059 ,
       -0.6128816 , -0.61979336, -0.6288633 , -0.64968634, -0.6670257 ,
       -0.6761421 , -0.67730856, -0.6816009 , -0.67545867, -0.6647226 ,
       -0.6719049 , -0.6619738 , -0.6471278 , -0.652049  , -0.64146125,
       -0.64391303, -0.6518209 , -0.67124003, -0.71799076, -0.70881754,
       -0.7226164 , -0.73732513, -0.7245811 , -0.71309674, -0.7026587 ,
       -0.7268016 , -0.76876825, -0.75947165, -0.76661104, -0.7662666 ,
       -0.76978624, -0.7542365 , -0.7629837 , -0.7993014 , -0.8069032 ,
       -0.80255187, -0.80604035, -0.8206661 , -0.817362  , -0.81978494,
       -0.83180475, -0.81759435, -0.8207392 , -0.82689744, -0.83627135,
       -0.8303414 , -0.8460805 , -0.85512656, -0.8524952 , -0.84508985,
       -0.83099365, -0.8284293 , -0.8292149 , -0.83307105, -0.8172412 ,
       -0.82091004, -0.8238109 , -0.8224975 , -0.8299492 , -0.82

'mfcc'

(3318, 20)

...

(3318, 84)

array([-19.192596 , -18.357492 , -18.763315 , -19.366571 , -16.929043 ,
       -14.4525385, -14.049117 , -12.759987 , -14.402209 , -17.90481  ,
       -16.231672 , -15.364329 , -16.775997 , -17.590565 , -16.408525 ,
       -19.18176  , -19.740923 , -20.840792 , -21.245556 , -20.723444 ,
       -20.999773 , -25.08835  , -23.356092 , -25.212355 , -26.946945 ,
       -24.921827 , -25.974691 , -28.842213 , -28.154224 , -28.030458 ,
       -28.919643 , -29.377167 , -31.023067 , -31.180063 , -34.014633 ,
       -33.669846 , -34.84148  , -35.023224 , -36.85372  , -36.162834 ,
       -37.3165   , -37.896534 , -40.190365 , -40.017704 , -41.065468 ,
       -40.98337  , -41.63271  , -41.198174 , -40.882187 , -40.49316  ,
       -39.26483  , -39.1765   , -38.733685 , -39.23859  , -42.05581  ,
       -44.75164  , -46.034424 , -44.174923 , -45.603477 , -48.402534 ,
       -48.42529  , -48.000134 , -50.627438 , -51.769367 , -53.32704  ,
       -52.579865 , -53.69759  , -53.566704 , -54.655746 , -54.5

'id'

(3318,)

'XC428404.mp3'

## **SVM Model**

In [37]:
train_results = {}
val_results = {}

### **With all the features**

In [38]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
# A random forest model expect a 2D input of (n_samples, n_features). Since the input is 3D here, it will cause an error when passed through a RF,
# I need to flatten the training features from 3D to 2D... eg. (7105, 93 * 259)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)

training_features.shape

(12565, 244)

In [39]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(3318, 244)

Fit the model with training data

In [40]:
# model = SVC(kernel='rbf', C=4)
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)
model = KNeighborsClassifier(metric='manhattan')

param_grid = {'n_neighbors': [1, 5, 9, 15, 20,30,40]}

clf = GridSearchCV(
        model,
        param_grid,
        cv=5,
        scoring='accuracy',
        verbose=1,
        n_jobs=2,
    )

clf.fit(training_features, train_y)
display(clf.best_score_)
display(clf.best_params_)

model = model = KNeighborsClassifier(metric='manhattan', n_neighbors=clf.best_params_['n_neighbors'], weights='distance')
model.fit(training_features, train_y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


0.977158774373259

{'n_neighbors': 1}

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

In [41]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5557564798071127


In [42]:
train_yhat_result = model.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2060
           1       1.00      1.00      1.00      2477
           2       1.00      1.00      1.00      2203
           3       1.00      1.00      1.00       437
           4       1.00      1.00      1.00       218
           5       1.00      1.00      1.00       387
           6       1.00      1.00      1.00       167
           7       1.00      1.00      1.00       263
           8       1.00      1.00      1.00       196
           9       1.00      1.00      1.00       443
          10       1.00      1.00      1.00       191
          11       1.00      1.00      1.00       211
          12       1.00      1.00      1.00       378
          13       1.00      1.00      1.00       232
          14       1.00      1.00      1.00       188
          15       1.00      1.00      1.00      1133
          16       1.00      1.00      1.00     

In [43]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.60      0.55       455
           1       0.41      0.62      0.50       492
           2       0.65      0.48      0.55       889
           3       0.78      0.92      0.84       150
           4       0.17      0.04      0.07        67
           5       0.43      0.84      0.57        43
           6       0.12      0.29      0.17        24
           7       0.96      0.98      0.97        44
           8       0.42      0.64      0.50        50
           9       0.84      0.69      0.76       169
          10       0.49      0.40      0.44        53
          11       0.22      0.20      0.21        66
          12       0.89      0.58      0.70        59
          13       0.56      0.49      0.52        57
          14       0.27      0.53      0.36        38
          15       0.48      0.33      0.39       311
          16       0.91      0.96      0.94   

In [44]:
from sklearn.preprocessing import MinMaxScaler

In [45]:
# model = SVC(kernel='rbf', C=4)
scaler = MinMaxScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)


model = model = KNeighborsClassifier(metric='manhattan', n_neighbors=clf.best_params_['n_neighbors'], weights='distance')
model.fit(training_features, train_y)

KNeighborsClassifier(metric='manhattan', n_neighbors=1, weights='distance')

In [46]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['MinMaxScaler'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['MinMaxScaler'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.540084388185654


In [47]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.59      0.55       455
           1       0.41      0.61      0.49       492
           2       0.66      0.49      0.56       889
           3       0.74      0.89      0.81       150
           4       0.18      0.04      0.07        67
           5       0.45      0.81      0.58        43
           6       0.07      0.21      0.11        24
           7       0.91      0.98      0.95        44
           8       0.38      0.56      0.46        50
           9       0.78      0.67      0.72       169
          10       0.40      0.32      0.35        53
          11       0.21      0.23      0.22        66
          12       0.82      0.39      0.53        59
          13       0.51      0.42      0.46        57
          14       0.25      0.53      0.34        38
          15       0.46      0.28      0.34       311
          16       0.87      0.95      0.90   

In [48]:
from imblearn.over_sampling import SMOTE



In [49]:
# model = SVC(kernel='rbf', C=4)
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)

smote = SMOTE()
training_features_balanced, train_y_balanced = smote.fit_resample(training_features, train_y)

model = model = KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'], weights='distance')
model.fit(training_features_balanced, train_y_balanced)

KNeighborsClassifier(n_neighbors=1, weights='distance')

In [50]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['smote'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['smote'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5744424352019288


In [51]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.55      0.61      0.58       455
           1       0.44      0.63      0.52       492
           2       0.65      0.50      0.56       889
           3       0.87      0.95      0.91       150
           4       0.15      0.04      0.07        67
           5       0.45      0.86      0.59        43
           6       0.10      0.21      0.13        24
           7       0.93      0.98      0.96        44
           8       0.34      0.58      0.43        50
           9       0.86      0.70      0.77       169
          10       0.60      0.45      0.52        53
          11       0.22      0.18      0.20        66
          12       0.85      0.69      0.77        59
          13       0.63      0.67      0.65        57
          14       0.33      0.76      0.46        38
          15       0.50      0.29      0.37       311
          16       0.79      0.95      0.86   

# Review results from all models

In [52]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
1,MinMaxScaler,1.0,0.54
0,all_features,1.0,0.56
2,smote,1.0,0.57
